In [1]:
import tensorflow as tf
#占位符,声明一个float32类型张量
a=tf.placeholder(tf.float32)
b=tf.placeholder(tf.float32)
#建立图,加法图
y=tf.add(a,b)
y_1=tf.add(y,b)
#建立会话
sess=tf.Session()
#执行数据流图的操作
print(sess.run(y,{a:3,b:4}))
print(sess.run(y_1,{a:4,b:4}))
#关闭会话
sess.close()

7.0
12.0


# BP算法

In [ ]:
import numpy as np
#数据
w=[0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65]
b=[0.35,0.65]
I=[5,10]
out=[0.01,0.99]

In [ ]:
def bp1(I,w,b,o):
    #fp
    h1=sigmoid(w[0]*I[0]+w[1]*I[1]+b[0])
    h2=sigmoid(w[2]*I[0]+w[3]*I[1]+b[0])
    h3=sigmoid(w[4]*I[0]+w[5]*I[1]+b[0])
    
    o1=sigmoid(h1*w[6]+h2*w[8]+h3*w[10]+b[1])
    o2=sigmoid(h1*w[7]+h2*w[9]+h3*w[11]+b[1])
    
    e1=0.5*(o[0]-o1)**2
    e2=0.5*(o[1]-o2)**2
    
    #bp
    t1=-(o[0]-o1)*o1*(1-o1)
    t2=-(o[1]-o2)*o2*(1-o2)
    tw7=t1*h1
    tw9=t1*h2
    tw11=t1*h3
    
    tw8=t2*h1
    tw10=t2*h2
    tw12=t2*h3
    # b[1]
    # 输入层的求导
    tw1=t1*w7+t2*t7
    #...
    
    # b[0]
    
    # 更新
    w[0]=w[0]-0.5*tw1
    #......
    
    return w,b
for i in range(1000):
    w,b=bp1(I,w,b,out)

In [ ]:
# BP神经网络实现手写数字

In [22]:
# BP算法也叫做δ算法,求解W的算法
import numpy as np
import random

class Network:
    def __init__(self, sizes):
        """
        :param sizes: list类型，储存每层神经网络的神经元数目
                      譬如说：sizes = [2, 3, 2] 表示输入层有两个神经元、
                      隐藏层有3个神经元以及输出层有2个神经元
        """
        # 有几层神经网络
        self.num_layers = len(sizes)
        self.sizes = sizes
        # 除去输入层，随机产生每层中 y 个神经元的 biase 值（0 - 1）
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # 随机产生每条连接线的 weight 值（0 - 1）
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    # 向前传输（FreedForward）的代码
    def feedforward(self, a):
        """
        前向传输计算每个神经元的值
        :param a: 输入值
        :return: 计算后每个神经元的值
        """
        for b, w in zip(self.biases, self.weights):
            # 加权求和以及加上 biase
            a = sigmoid(np.dot(w, a) + b)
        return a

    # SGD
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """
        随机梯度下降
        :param training_data: 输入的训练集
        :param epochs: 迭代次数
        :param mini_batch_size: 小样本数量
        :param eta: 学习率
        :param test_data: 测试数据集
        """
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            # 搅乱训练集，让其排序顺序发生变化
            random.shuffle(training_data)
            # 按照小样本数量划分训练集
            mini_batches = [
                training_data[k:k + mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                # 根据每个小样本来更新 w 和 b，代码在下一段
                self.update_mini_batch(mini_batch, eta)
            # 输出测试每轮结束后，神经网络的准确度
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    # backprop 方法得到的偏导数更新 w 和 b 的值

    def update_mini_batch(self, mini_batch, eta):
        """
        更新 w 和 b 的值
        :param mini_batch: 一部分的样本
        :param eta: 学习率
        """
        # 根据 biases 和 weights 的行列数创建对应的全部元素值为 0 的空矩阵
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            # 根据样本中的每一个输入 x 的其输出 y，计算 w 和 b 的偏导数
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            # 累加储存偏导值 delta_nabla_b 和 delta_nabla_w
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        # 更新根据累加的偏导值更新 w 和 b，这里因为用了小样本，
        # 所以 eta 要除于小样本的长度
        self.weights = [w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]

    # BP 算法的实现
    def backprop(self, x, y):
        """
        :param x:
        :param y:
        :return:
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # 前向传输
        activation = x
        # 储存每层的神经元的值的矩阵，下面循环会 append 每层的神经元的值
        activations = [x]
        # 储存每个未经过 sigmoid 计算的神经元的值
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # 求 δ 的值
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        # 乘于前一层的输出值
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            # 从倒数第 l 层开始更新，-l 是 python 中特有的语法表示从倒数第 l 层开始计算
            # 下面这里利用 l+1 层的 δ 值来计算 l 的 δ 值
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)
        # evaluate 的实现

    def evaluate(self, test_data):
        # 获得预测结果
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        # 返回正确识别的个数
        return sum(int(x == y) for (x, y) in test_results)


def sigmoid(y):
    return 1 / (1 + np.exp(-y))

In [35]:
from __future__ import print_function,division
import pickle
import gzip
import numpy as np

#从数据集中载入数据
def load_data():
    file=gzip.open('mnist.pkl.gz','rb')
    training_data,validation_data,test_data=pickle.load('mnist.pkl')
    file.close()
    return training_data,validation_data,test_data

#改编数据集的格式
def data_wrapper():
    tr_d,va_d,te_d=load_data()
    #训练集
    training_inputs=[np.reshape(x,(784,1)) for x in tr_d[0]]
    training_labels=[vectorized_label(x) for x in tr_d[1]]
    training_data=zip(training_inputs,training_labels)

    #验证集
    validation_inputs=[np.reshape(x,(784,1)) for x in va_d[0]]
    validation_data=zip(validation_inputs,va_d[1])

    #测试集
    test_inputs=[np.reshape(x,(784,1)) for x in te_d[0]]
    test_data=zip(test_inputs,te_d[1])

    return training_data,validation_data,test_data

def vectorized_label(j):
    #形状为10行1列
    e=np.zeros((10,1))
    e[j]=1.0
    return e

In [ ]:
training_data, validation_data, test_data = data_wrapper()
net = network.Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data = test_data)